In [1]:
import pandas as pd

In [52]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import  mean_squared_error
import numpy as np

In [26]:
cdsSpread_data = pd.read_stata("C:\\Users\\Arpita\\Desktop\\Book\\Fall 2018\\Machine Learning\\Assignment\\cds_spread5y_2001_2016.dta")


In [27]:
cdsSpread_data.shape

(169043, 9)

In [28]:
cdsSpread_data.dtypes

gvkey                    object
mdate            datetime64[ns]
redcode                  object
tier                     object
ticker                   object
shortname                object
docclause                object
spread5y                float64
impliedrating            object
dtype: object

In [29]:
cdsSpread_data.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB


In [30]:
#Seperate date based on month & years and create quarters accordingly 
cdsSpread_data['Date'] = pd.to_datetime(cdsSpread_data['mdate'])
cdsSpread_data['Month']= cdsSpread_data['Date'].dt.month
cdsSpread_data['Year']=cdsSpread_data['Date'].dt.year

cdsSpread_data['quarter']='4'


In [31]:
cdsSpread_data.loc[cdsSpread_data['Month']>9,"quarter"]=4
cdsSpread_data.loc[(cdsSpread_data['Month']>6) & (cdsSpread_data['Month']<=9),"quarter"]=3
cdsSpread_data.loc[(cdsSpread_data['Month']>3) & (cdsSpread_data['Month']<=6),"quarter"]=2
cdsSpread_data.loc[(cdsSpread_data['Month'])<=3,"quarter"]=1

In [32]:
cdsSpread_data['gvkey'] = cdsSpread_data['gvkey'].astype(float)
cdsSpread_data['quarter'] = cdsSpread_data['quarter'].astype(float)
cdsSpread_data['Year'] = cdsSpread_data['Year'].astype(float)

In [33]:
qtr_Mdata = pd.read_csv("C:\\Users\\Arpita\\Downloads\\Quarterly Merged CRSP-Compustat.csv", low_memory = False)

In [34]:
qtr_Mdata.shape

(58928, 687)

In [35]:
qtr_Mdata.dtypes

GVKEY          int64
LINKPRIM      object
LIID          object
LINKTYPE      object
LPERMNO        int64
LPERMCO        int64
LINKDT        object
LINKENDDT     object
datadate      object
fyearq         int64
fqtr           int64
fyr            int64
indfmt        object
consol        object
popsrc        object
datafmt       object
tic           object
cusip         object
conm          object
acctchgq      object
acctstdq      object
adrrq        float64
ajexq        float64
ajpq         float64
bsprq         object
compstq       object
curcdq        object
curncdq       object
currtrq      float64
curuscnq     float64
              ...   
addzip        object
busdesc       object
city          object
conml         object
county        object
dlrsn        float64
ein           object
fax           object
fyrc           int64
ggroup         int64
gind           int64
gsector        int64
gsubind        int64
idbflag       object
incorp        object
loc           object
naics        

In [36]:
qtr_Mdata.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyearq,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2001,2000,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
1,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,06/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
2,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,09/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
3,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,12/31/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
4,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2002,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN


In [37]:
qtr_Mdata=qtr_Mdata.rename(columns = {'GVKEY':'gvkey'})
qtr_Mdata=qtr_Mdata.rename(columns = {'datadate':'mdate'})

In [49]:
qtr_Mdata['mdate'].unique()

array(['03/31/2001', '06/30/2001', '09/30/2001', '12/31/2001',
       '03/31/2002', '06/30/2002', '09/30/2002', '12/31/2002',
       '03/31/2003', '06/30/2003', '09/30/2003', '12/31/2003',
       '03/31/2004', '06/30/2004', '09/30/2004', '12/31/2004',
       '03/31/2005', '06/30/2005', '09/30/2005', '12/31/2005',
       '03/31/2006', '06/30/2006', '09/30/2006', '12/31/2006',
       '03/31/2007', '06/30/2007', '09/30/2007', '12/31/2007',
       '03/31/2008', '06/30/2008', '09/30/2008', '12/31/2008',
       '03/31/2009', '06/30/2009', '09/30/2009', '12/31/2009',
       '03/31/2010', '06/30/2010', '09/30/2010', '12/31/2010',
       '03/31/2011', '06/30/2011', '09/30/2011', '12/31/2011',
       '12/31/2013', '03/31/2014', '06/30/2014', '09/30/2014',
       '12/31/2014', '03/31/2015', '06/30/2015', '09/30/2015',
       '12/31/2015', '03/31/2016', '06/30/2016', '09/30/2016',
       '12/31/2016', '03/31/2017', '03/31/2012', '06/30/2012',
       '09/30/2012', '12/31/2012', '03/31/2013', '06/30

In [38]:
#Seperate date based on month & years and create quarters accordingly 
qtr_Mdata['Date'] = pd.to_datetime(qtr_Mdata['mdate'])
qtr_Mdata['Month']= qtr_Mdata['Date'].dt.month
qtr_Mdata['Year']=qtr_Mdata['Date'].dt.year

qtr_Mdata['quarter']='4'


In [39]:
qtr_Mdata.loc[qtr_Mdata['Month']>9,"quarter"]=4
qtr_Mdata.loc[(qtr_Mdata['Month']>6) & (qtr_Mdata['Month']<=9),"quarter"]=3
qtr_Mdata.loc[(qtr_Mdata['Month']>3) & (qtr_Mdata['Month']<=6),"quarter"]=2
qtr_Mdata.loc[(qtr_Mdata['Month'])<=3,"quarter"]=1

In [40]:
qtr_Mdata['gvkey'] = qtr_Mdata['gvkey'].astype(float)
qtr_Mdata['quarter'] = qtr_Mdata['quarter'].astype(float)
qtr_Mdata['Year'] = qtr_Mdata['Year'].astype(float)

In [41]:
qtr_Mdata['quarter'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
Name: quarter, dtype: float64

In [42]:
data = pd.merge(cdsSpread_data[1:200], qtr_Mdata[1:200], on=['gvkey', 'quarter', 'Year'])

In [50]:
#Use only numerical records 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data = data.select_dtypes(include=numerics)

#Pre processing data: replace NA with median 
data=data.fillna(data.median())
data=data.dropna(axis=1, how='any')

#Split the dataset 
TestData = data[(data['Year'] >= 2010) & (data['Year'] <= 2018)]

#Initialze X and Y
X_test= TestData.drop('spread5y', axis=1)
y_test=TestData['spread5y']

X_test= X_test.drop('Month_x', axis=1)
X_test= X_test.drop('Month_y', axis=1)
X_test= X_test.drop('quarter', axis=1)
X_test= X_test.drop('Year', axis=1)
X_test= X_test.drop('gvkey', axis=1)

TrainData=data[(data['Year'] < 2010)]

#splitting x and y for test data
X_train= TrainData.drop('spread5y', axis=1)
y_train=TrainData['spread5y']
X_train= X_train.drop('Month_x', axis=1)
X_train= X_train.drop('Month_y', axis=1)
X_train= X_train.drop('quarter', axis=1)
X_train= X_train.drop('Year', axis=1)
X_train= X_train.drop('gvkey', axis=1)

In [51]:
randforst = RandomForestRegressor(n_estimators=50) 
randforst.fit(X_train, y_train)
#randforst.score(X_test, y_test)
randforst_Pred=randforst.predict(X_test)

In [57]:
F_imp=randforst.feature_importances_
F_imp = pd.DataFrame(randforst.feature_importances_,index = X_train.columns,columns=['imp']).sort_values('imp',ascending=False)
newfeatures=F_imp.iloc[:50,:]
newfeatures=newfeatures.index.tolist()

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_squared_error(y_test,randforst_Pred)
mean_absolute_percentage_error(y_test, randforst_Pred)


41.23385672821293

In [60]:
# Random Forest with 100 estimators 
newtrain_x = X_train[list(newfeatures)]
newtest_x = X_test[list(newfeatures)]

regressor_100 = RandomForestRegressor(n_estimators = 100,max_depth = 3)
regressor_100.fit(newtrain_x,y_train)
pred_100 = regressor_100.predict(newtest_x)
print('Mean Accuracy_100:', regressor_100.score(newtest_x,y_test))
errors_100 = abs(pred_100 - y_test)
mape_100 = 100 * (errors_100 / y_test)
# Calculate and display accuracy
accuracy_100 = 100 - np.mean(mape_100)
print('Accuracy_100:', round(accuracy_100, 2), '%.')

Mean Accuracy_100: -0.41954668676032436
Accuracy_100: 61.67 %.


In [61]:
# Random Forest with 200 estimators 
regressor_200 = RandomForestRegressor(n_estimators = 200,max_depth = 3)
regressor_200.fit(newtrain_x,y_train)
pred_200 = regressor_200.predict(newtest_x)
print('Mean Accuracy_200:', regressor_200.score(newtest_x,y_test))
errors_200 = abs(pred_200 - y_test)
mape_200 = 200 * (errors_200 / y_test)
# Calculate and display accuracy
accuracy_200 = 200 - np.mean(mape_200)
print('Accuracy_200:', round(accuracy_200, 2), '%.')

Mean Accuracy_200: -0.3155484038252572
Accuracy_200: 137.33 %.


In [62]:
# Random Forest with 500 estimators 
regressor_500 = RandomForestRegressor(n_estimators = 500,max_depth = 3)
regressor_500.fit(newtrain_x,y_train)
pred_500 = regressor_500.predict(newtest_x)
print('Mean Accuracy_500:', regressor_500.score(newtest_x,y_test))
errors_500 = abs(pred_500 - y_test)
mape_500 = 500 * (errors_500 / y_test)
# Calculate and display accuracy
accuracy_500 = 500 - np.mean(mape_500)
print('Accuracy_500:', round(accuracy_500, 2), '%.')

Mean Accuracy_500: -0.36143055015640124
Accuracy_500: 333.21 %.


In [63]:
# Random Forest with 1000 estimators 
regressor_1000 = RandomForestRegressor(n_estimators = 1000,max_depth = 3)
regressor_1000.fit(newtrain_x,y_train)
pred_1000 = regressor_1000.predict(newtest_x)
print('Mean Accuracy_1000:', regressor_1000.score(newtest_x,y_test))
errors_1000 = abs(pred_1000 - y_test)
mape_1000 = 1000 * (errors_1000 / y_test)
# Calculate and display accuracy
accuracy_1000 = 1000 - np.mean(mape_1000)
print('Accuracy_1000:', round(accuracy_1000, 2), '%.')

Mean Accuracy_1000: -0.2905437252016432
Accuracy_1000: 660.95 %.


In [80]:
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import xgboost

In [73]:
# Gradient Boost with 100 estimators 
GB_100 = ensemble.GradientBoostingRegressor(n_estimators = 100, max_depth = 3)
GB_100.fit(newtrain_x, y_train)
mse_100 = mean_squared_error(y_test, GB_100.predict(newtest_x))
print("MSE_100: %.4f" % mse_100)

MSE_100: 0.0049


In [74]:
# Gradient Boost with 200 estimators 
GB_200 = ensemble.GradientBoostingRegressor(n_estimators = 200, max_depth = 3)
GB_200.fit(newtrain_x, y_train)
mse_200 = mean_squared_error(y_test, GB_200.predict(newtest_x))
print("MSE_200: %.4f" % mse_200)

MSE_200: 0.0048


In [75]:
# Gradient Boost with 500 estimators 
GB_500 = ensemble.GradientBoostingRegressor(n_estimators = 500, max_depth = 3)
GB_500.fit(newtrain_x, y_train)
mse_500 = mean_squared_error(y_test, GB_500.predict(newtest_x))
print("MSE_500: %.4f" % mse_500)

MSE_500: 0.0047


In [76]:
# Gradient Boost with 1000 estimators 
GB_1000 = ensemble.GradientBoostingRegressor(n_estimators = 1000, max_depth = 3)
GB_1000.fit(newtrain_x, y_train)
mse_1000 = mean_squared_error(y_test, GB_1000.predict(newtest_x))
print("MSE_1000: %.4f" % mse_1000)


MSE_1000: 0.0044


In [81]:
# XG Boost with 100 estimators 
xgb_100 = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_100.fit(newtrain_x, y_train)
xgbmse_100 = mean_squared_error(y_test, xgb_100.predict(newtest_x))
print("xgbMSE1: %.4f" % xgbmse_100)

xgbMSE1: 0.0044


In [82]:
# XG Boost with 200 estimators 
xgb_200 = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=3)
xgb_200.fit(newtrain_x, y_train)
xgbmse_200 = mean_squared_error(y_test, xgb_200.predict(newtest_x))
print("xgbMSE1: %.4f" % xgbmse_200)

xgbMSE1: 0.0044


In [83]:
# XG Boost with 500 estimators 
xgb_500 = xgboost.XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=3)
xgb_500.fit(newtrain_x, y_train)
xgbmse_500 = mean_squared_error(y_test, xgb_500.predict(newtest_x))
print("xgbMSE1: %.4f" % xgbmse_500)

xgbMSE1: 0.0044


In [84]:
# XG Boost with 1000 estimators 
xgb_1000 = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=3)
xgb_1000.fit(newtrain_x, y_train)
xgbmse_1000 = mean_squared_error(y_test, xgb_1000.predict(newtest_x))
print("xgbMSE1: %.4f" % xgbmse_1000)

xgbMSE1: 0.0044
